In [1]:
import numpy as np
import pandas as pd
import random

As mentioned in read me 
Default number of hidden states is 5
Default number is also 5, all the elements in visible sequence is between [0,4]
If you have raw sequence on which HMM has to be applied first get number number of unique hidden states and unique visible states and map states and visible states as follows

In [48]:
sequence=[0,1,2,3,1,1,1,1,1,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,3,3,3,1,1,1,1,1,1,3,2,1,0,4,3,2,1,2,3,4,3,2]
sequence=np.array(sequence)
intial=[0.05,0.5,0.3,0.05,0.1]
intial=np.array(intial)
print(len(sequence))

60


In [57]:
### .............................................................Implementation of hidden markov model.................................................................
## 3 Taks to train HMM
  ## 1) Forward Algorithm
  ## 2) Backward Algorithm
  ## 3) Forward Backward Algorithm

class HMM:
    def __init__(self,sequence,initial_matrix,number_states=5,number_output=5,n_epochs=30):
        self.seq=sequence
        self.tras=np.random.uniform(0,1,(number_states,number_states))  ## Transition Matrix 
        #self.tras=np.zeros(shape=(number_states,number_states))        ## Initialized with random number drawn from uniform (0,1)
        self.emiss=np.random.uniform(0,1,(number_states,number_output)) ## Emission Matrix
        #self.emiss=np.zeros(shape=(number_states,number_output))
        print(len(self.seq))
        self.iprob=initial_matrix         ## Intial Probabilty matrix (Pie)
        
        self.states=number_states         ## Number of hidden States
        
        self.out=number_output            ## Number of Visible States
        
        self.epochs=n_epochs              ## Number of Iterations default value is 30
        
        self.BDP=np.zeros(shape=(number_states,len(self.seq)))  ## 2d numpy array to store alpha(state,time)
        
        self.FDP=np.zeros(shape=(number_states,len(self.seq)))   ## 2d numpy array to store  Beta(state,time)
        
        self.n_epochs=n_epochs
        
    def forward(self):                                                          
        nstates=self.states                                                       
        nlength=len(self.seq)                                                     
        maxi=0
        index=0                                                                     
        for j in range(nlength):
            for i in range(nstates):                         
                if j==0:
                    self.FDP[i][j]=self.iprob[i]*self.emiss[i][self.seq[j]]
                else:
                    for k in range(nstates):
                        self.FDP[i][j]+=(self.FDP[k][j-1]*self.tras[k][i]*self.emiss[i][self.seq[j]])
                    if j==nlength-1:
                        if(self.FDP[i][j]>=maxi):
                            maxi=self.FDP[i][j]
                            index=i
        tp=0.0
        for i in range(nstates):
            tp+=self.FDP[i][nlength-1]
        return tp,maxi,index
    ## Dynamic algorithm Space complexity O(states*length of Sequence) Time complexity O(states*states*length of sequence)
    
    def decoding(self):                                                           
        nstates=self.states                                                       
        nlength=len(self.seq)                                                     
        tp,maxi,index=self.forward()
        hstates=np.zeros(shape=(nlength))
        ## Backtracking to get the most probable hidden states for each time step
        
        for j in range(nlength-1,-1,-1):
            hstates[j]=index
            if(j==0):
                break
            for i in range(nstates):
                self.FDP[index][j]==self.FDP[i][j-1]*self.tras[i][index]*self.emiss[index][self.seq[j]]
                index=i
                break
        return hstates
        
    def backward(self):
        ## Algo for calculating B(state,time)
        nstates=self.states                                                 
        nlength=len(self.seq)                                                     
        for j in range(nlength-1,-1,-1):
            for i in range(nstates):
                if(j==nlength-1):
                    self.BDP[i][j]=1.0
                else:
                    for k in range(nstates):
                        self.BDP[i][j]+=(self.tras[i][k]*self.emiss[k][self.seq[j+1]]*self.BDP[k][j+1])
                   
                
    def not_quite(self,time,curr_state,next_state):
        #self.forward()
        #self.backward()
        ## Algo for calculating mu as per paper
        nstates=self.states                                                 
        nlength=len(self.seq)
        deno=0
        for i in range(nstates):
            deno+=self.FDP[i][time]*self.BDP[i][time]
        numo=self.FDP[curr_state][time]*self.tras[curr_state][next_state]*self.emiss[next_state][self.seq[time+1]]*self.BDP[next_state][time+1]
        return numo/deno
    
    
    def corrected_transition(self):
        ## Corrected transiton matrix 
        nstates=self.states                                                 
        nlength=len(self.seq)
        
        for i in  range(nstates):
            for j in range(nstates):
                numo=0
                deno=0
                for time in range(nlength-1):
                    numo+=self.not_quite(time,i,j)
                    for state in range(nstates):
                        deno+=self.not_quite(time,i,state)
                self.tras[i][j]=numo/deno
    
    
    def not_quite2(self,time,state):
        nstates=self.states                                                 
        nlength=len(self.seq)
        deno=0
        for st in range(nstates):
            deno+=self.FDP[st][time]*self.BDP[st][time]
        return self.FDP[state][time]*self.BDP[state][time]/deno
    
    
    def corrected_emmision(self):
        # Algorithm for calculating gamma as per pdf
        nstates=self.states                                                 
        nlength=len(self.seq)
        nout=self.out
        for states in (range(nstates)):
            for out in (range(nout)):
                numo=0
                deno=0
                for time in range(nlength):
                    if(self.seq[time]==out):
                        numo+=self.not_quite2(time,states)
                    deno+=self.not_quite2(time,states)
                self.emiss[states][out]=numo/deno
                
    def display(self):
        print("Transition Matrix")
        print(self.tras)
        print(" ")
        print(" ")
        print("Emmision Matrix")
        print(self.emiss)
            
    def train(self):
        for epoch in range(self.n_epochs):               ## Iterating fot number of epochs
            print("EPOCH NO : {} TRANING".format(epoch))
            self.forward()                               ## calculate corrected alpha 
            self.backward()                              ## calculate corrected Beta
            self.corrected_transition()                  ## Corrected Transition matrix
            self.corrected_emmision()                    ## Corrected Emission matrix
            print("Transition Matrix")
            print(self.tras)
            print("Emmission Matrix")
            print(self.emiss)
            print("Traning Epoch Done")
            print(" ")
            print(" ")

In [58]:
# deno+=self.not_quite(time,i,states)

In [59]:
hm=HMM(sequence,intial,n_epochs=50)

60


In [60]:
hm.train()

EPOCH NO : 0 TRANING
Transition Matrix
[[0.18750083 0.08608146 0.24469218 0.49282813 0.12967306]
 [0.09849265 0.00853916 0.43769477 0.62997342 0.04729197]
 [0.03185276 0.15631327 0.39363479 0.50715755 0.20959229]
 [0.11956104 0.10029509 0.33230949 0.57308665 0.23431766]
 [0.16857367 0.11613471 0.36006557 0.47154798 0.20658681]]
Emmission Matrix
[[0.55586802 0.15053917 0.09432519 0.15724073 0.04202688]
 [0.46394385 0.28198187 0.10580199 0.12613534 0.02213695]
 [0.34371706 0.40360709 0.20479451 0.01843117 0.02945016]
 [0.35025488 0.09555512 0.26247835 0.2517446  0.03996705]
 [0.36821207 0.24140484 0.32051739 0.04485921 0.02500649]]
Traning Epoch Done
 
 
EPOCH NO : 1 TRANING
Transition Matrix
[[0.14149274 0.07862934 0.21831224 0.4940614  0.11820698]
 [0.06121589 0.0070797  0.39866288 0.53690631 0.04352123]
 [0.0179118  0.11647297 0.33460925 0.42444786 0.18362048]
 [0.06848958 0.07096882 0.23993587 0.52931662 0.19372685]
 [0.10038431 0.08941811 0.2956812  0.42809182 0.18131152]]
Emmission

Transition Matrix
[[2.33208633e-02 1.06851690e-01 3.04983259e-01 4.01867067e-01
  3.82315014e-02]
 [7.49273938e-04 1.84100631e-03 9.81824614e-01 1.23275503e-02
  3.81365747e-03]
 [1.47771690e-04 3.30765393e-02 9.33213698e-01 1.53320457e-02
  2.12196033e-02]
 [4.36025448e-03 5.65541654e-03 7.07623584e-03 9.05162478e-01
  7.93893969e-02]
 [6.83848424e-03 5.20932018e-03 9.57884133e-03 8.11172748e-01
  1.72817395e-01]]
Emmission Matrix
[[0.22279192 0.13596688 0.14980257 0.30770774 0.18373089]
 [0.3217257  0.27768036 0.06967573 0.31611686 0.01480135]
 [0.02319598 0.74326615 0.15765098 0.06281161 0.01307528]
 [0.54909771 0.02333103 0.22987312 0.15824637 0.03945177]
 [0.43067952 0.1120219  0.38641764 0.04352692 0.02735402]]
Traning Epoch Done
 
 
EPOCH NO : 17 TRANING
Transition Matrix
[[1.85054902e-02 1.67854662e-01 4.64446485e-01 2.07698983e-01
  2.08739241e-02]
 [3.18144887e-04 1.38980005e-03 9.92531608e-01 4.57298886e-03
  1.64816682e-03]
 [8.73887073e-05 3.03716420e-02 9.45171855e-01 9.2

Transition Matrix
[[8.51326508e-01 1.12524881e-01 3.65830518e-02 4.55268016e-11
  3.38390459e-10]
 [5.83462275e-07 4.84626200e-06 9.99998423e-01 1.48636045e-12
  1.14813256e-11]
 [2.04052447e-08 8.93745755e-05 9.34503054e-01 1.46347162e-08
  6.46327382e-02]
 [3.65109102e-03 8.43592352e-06 7.63202776e-07 9.69570931e-01
  2.96622927e-02]
 [2.63945551e-01 1.43517953e-05 1.07467275e-07 1.20856016e-01
  5.86217077e-01]]
Emmission Matrix
[[0.00189954 0.01055075 0.06552303 0.7226313  0.19939538]
 [0.491328   0.10825284 0.0516601  0.3415353  0.00722376]
 [0.0008806  0.70049282 0.16870688 0.12497049 0.00494921]
 [0.82659664 0.00516729 0.09242469 0.04694836 0.02886302]
 [0.09064899 0.0576779  0.81991848 0.01439172 0.01736291]]
Traning Epoch Done
 
 
EPOCH NO : 31 TRANING
Transition Matrix
[[8.16465142e-01 1.19761541e-01 6.95858874e-02 1.03311048e-11
  1.28099919e-10]
 [7.55899857e-08 1.35310897e-06 9.99999612e-01 6.90036638e-14
  1.32993239e-12]
 [5.71156529e-09 5.16075479e-05 9.37656514e-01 3.3

Transition Matrix
[[5.98673315e-01 2.61226901e-04 4.04262249e-01 1.96452007e-28
  4.41422145e-18]
 [2.43277319e-21 4.46945959e-18 1.00000000e+00 1.55215313e-41
  4.32574507e-25]
 [1.48910906e-13 1.38487013e-10 8.12653644e-01 5.34825006e-19
  1.89331325e-01]
 [3.12613900e-16 2.25151808e-21 6.36178192e-20 9.70016341e-01
  3.00696062e-02]
 [1.89761327e-01 2.21602699e-14 7.45214579e-18 1.34033096e-01
  6.81097789e-01]]
Emmission Matrix
[[8.92307371e-06 1.55948034e-03 3.19584208e-02 7.07286477e-01
  2.59186698e-01]
 [8.72315870e-01 2.25846057e-02 2.80511816e-02 7.58282825e-02
  1.22006024e-03]
 [8.53921656e-07 6.97638086e-01 1.73246034e-01 1.28543474e-01
  5.71552577e-04]
 [9.81392624e-01 1.88246066e-04 1.19948020e-02 4.52490307e-03
  1.89942495e-03]
 [1.04603125e-01 3.58348691e-02 8.54440332e-01 3.79993514e-03
  1.32173935e-03]]
Traning Epoch Done
 
 
EPOCH NO : 45 TRANING
Transition Matrix
[[5.82552782e-01 5.01504628e-05 4.24066874e-01 7.23193259e-30
  1.16992625e-18]
 [8.49590031e-23 1.5

In [61]:
hm.display()

Transition Matrix
[[5.07363951e-01 1.27487478e-08 4.99825221e-01 1.05430372e-35
  5.39935704e-21]
 [9.00742448e-29 5.57644131e-26 1.00000000e+00 2.48089620e-57
  6.83039229e-30]
 [2.34260863e-15 2.19389976e-15 8.12262217e-01 1.86874521e-24
  1.87483090e-01]
 [1.46625237e-22 1.49767108e-27 5.07373768e-25 9.64460261e-01
  3.55159708e-02]
 [1.81677687e-01 1.46810875e-20 2.72881709e-21 1.32518355e-01
  6.83852745e-01]]
 
 
Emmision Matrix
[[5.44864006e-07 1.14041231e-03 2.94902619e-02 7.09714949e-01
  2.59653832e-01]
 [9.40600249e-01 1.61041222e-02 1.33270128e-02 2.98295417e-02
  1.39074549e-04]
 [2.07340266e-08 6.99632753e-01 1.71977486e-01 1.28302007e-01
  8.77339764e-05]
 [9.94206524e-01 1.35530634e-04 4.70563950e-03 7.79897589e-04
  1.72408325e-04]
 [1.06401667e-01 3.38543978e-02 8.58212247e-01 1.25908581e-03
  2.72601610e-04]]
